import llamaIndex and openAI model and libraries

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings 
from llama_index.llms.openai import OpenAI 
from llama_index.core.agent import FunctionAgent
from llama_index.embeddings.openai import OpenAIEmbedding
from duckduckgo_search import DDGS 
from llama_index.core.tools import QueryEngineTool
import os
os.environ["OPENAI_API_KEY"] = "YOUR-API-KEY"


#configure 
Settings.llm = OpenAI(model="gpt-4o-mini") 
Settings.embed_model = OpenAIEmbedding()


web search

In [33]:
from llama_index.core.tools import FunctionTool

search_tool = FunctionTool.from_defaults(
    fn=DDGS,
    name="duckduckgo_search",
    description="Performs a DuckDuckGo web search to find relevant information based on a query.",
)

document search

In [34]:
documents = SimpleDirectoryReader(input_dir="./applications").load_data()
index = VectorStoreIndex.from_documents(documents)
index.storage_context.persist(persist_dir="./storage")

query_engine = index.as_query_engine()
file_research_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    description="A tool for answering questions about the provided documents."
)


2025-09-11 16:36:06,326 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


define agent

In [35]:
agent = FunctionAgent(
    tools=[search_tool, file_research_tool],
    llm=Settings.llm,
    verbose=True,
    system_prompt="You are a helpful research assistant to answer questions about Cambio Labs. Use the DuckDuckGo search tool and file_research_tool to find information and answer the questions in regards to cambio labs thoroughly."
)



/opt/anaconda3/lib/python3.12/inspect.py:586: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  value = getter(object, key)
/opt/anaconda3/lib/python3.12/inspect.py:586: PydanticDeprecatedSince20: The `__fields_set__` attribute is deprecated, use `model_fields_set` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  value = getter(object, key)
/opt/anaconda3/lib/python3.12/inspect.py:586: PydanticDeprecatedSince211: Accessing the 'model_computed_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  value = getter(object, key)
/opt/anaconda3/lib/python3.12/inspect.py:586: PydanticDeprecatedSince211: Accessing the

running agent

In [39]:
import asyncio

research_query = "How would you describe your organization in three words?"
response = await query_engine.aquery(research_query)
print(response)

2025-09-11 16:38:51,602 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-09-11 16:38:52,970 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Empowering, Innovative, Inclusive.
